In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import os
from keras.models import load_model



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Set up paths to the dataset
train_dir = '/content/drive/MyDrive/Colab Notebooks/Walter/training'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Walter/testing'

# Define the number of classes
num_classes = 103 # Adjust the number of classes

# Define batch sizes for training and validation
train_batch_size = 16
validation_batch_size = 16

# Data Augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# No augmentation for validation data (only rescaling)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Connect the Image Data Generators to your dataset
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=train_batch_size,
    class_mode='sparse'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=validation_batch_size,
    class_mode='sparse'
)


Found 7309 images belonging to 103 classes.
Found 1895 images belonging to 103 classes.


In [4]:
print(train_generator.class_indices)

{'Aloevera': 0, 'Alpinia Galanga': 1, 'Amaranthus Viridis': 2, 'Amla': 3, 'Amruthaballi': 4, 'Arali': 5, 'Artocarpus Heterophyllus': 6, 'Astma_weed': 7, 'Azadirachta Indica': 8, 'Badipala': 9, 'Balloon_Vine': 10, 'Bamboo': 11, 'Basella Alba': 12, 'Beans': 13, 'Betel': 14, 'Bhrami': 15, 'Brassica Juncea': 16, 'Bringaraja': 17, 'Caricature': 18, 'Carissa Carandas': 19, 'Castor': 20, 'Catharanthus': 21, 'Chakte': 22, 'Chilly': 23, 'Citron lime (herelikai)': 24, 'Citrus Limon': 25, 'Coffee': 26, 'Common rue(naagdalli)': 27, 'Coriender': 28, 'Curry': 29, 'Doddpathre': 30, 'Ekka': 31, 'Eucalyptus': 32, 'Ficus Auriculata': 33, 'Ficus Religiosa': 34, 'Ganigale': 35, 'Ganike': 36, 'Gasagase': 37, 'Ginger': 38, 'Globe Amarnath': 39, 'Henna': 40, 'Hibiscus': 41, 'Hibiscus Rosa-sinensis': 42, 'Honge': 43, 'Insulin': 44, 'Jasmine': 45, 'Jasminum': 46, 'Kambajala': 47, 'Kasambruga': 48, 'Kohlrabi': 49, 'Lantana': 50, 'Lemongrass': 51, 'Malabar_Nut': 52, 'Malabar_Spinach': 53, 'Mangifera Indica': 54,

In [5]:

# Load MobileNetV2 pre-trained on ImageNet data but without the top classification layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base_model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of MobileNetV2
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# View the structure of the model
model.summary()

9406464/9406464 [==============================] - 0s 0us/step


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [8]:


# Define the number of epochs
epochs = 10  # You can change this value according to your needs

# Set the path to the folder on Google Drive
drive_folder_path = '/content/drive/MyDrive/Colab Notebooks/Walter/Epics'
model_file_path = os.path.join(drive_folder_path, 'MobileNet_2_model.h5')

# Check if the model file exists
if os.path.exists(model_file_path):
    # Load the model if the .h5 file is found
    print("Model file found. Loading the model...")
    model = load_model(model_file_path)
else:
    # Add callbacks for saving the best model and early stopping
    checkpoint = ModelCheckpoint(model_file_path, verbose=1, save_best_only=True)
    early_stop = EarlyStopping(patience=10, verbose=1)

    # Train the model
    print("Model file not found. Training the model...")
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_batch_size,
        epochs=epochs,
        callbacks=[checkpoint, early_stop]
    )


Model file found. Loading the model...


In [9]:
test_loss, test_accuracy = model.evaluate(validation_generator, steps=len(validation_generator))
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


119/119 [==============================] - 898s 8s/step - loss: 1.2646 - accuracy: 0.8037
Test Loss: 1.2646373510360718, Test Accuracy: 0.803693950176239


In [13]:
import os
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input

def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    processed_image = preprocess_input(img_array)
    return processed_image

def load_model_and_predict(image_path, model_path='/content/drive/MyDrive/Colab Notebooks/Walter/Epics/MobileNet_2_model.h5'):
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found at {model_path}. Please ensure the model is trained and saved at the specified path.")

    model = load_model(model_path)
    processed_image = load_and_preprocess_image(image_path)
    predictions = model.predict(processed_image)

    # Assuming train_generator is defined globally or passed as an argument to this function
    class_mapping = train_generator.class_indices
    class_labels = list(class_mapping.keys())

    predicted_class_index = np.argmax(predictions)
    predicted_class_label = class_labels[predicted_class_index]

    result = {"prediction": predicted_class_label, "confidence": float(predictions[0][predicted_class_index])}
    return result

# Example usage
image_path = '/content/drive/MyDrive/Colab Notebooks/Walter/Test_imagesssssss/Neem.png'
prediction_result = load_model_and_predict(image_path)
print(prediction_result)


1/1 [==============================] - 1s 723ms/step
{'prediction': 'Azadirachta Indica', 'confidence': 0.9906141757965088}


In [ ]:
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()
plot_model()
